# Advent of code 2024. An approach for solving day 12
---
My colleague at work invited me to *day 12* of the [Advent of Code 2024 challenge](https://adventofcode.com/2024/day/12). Quite interesting... This notebook reflects only the lastest, working code finished 03-01-2025. There were two attempts before....

## Some remarks
* This is my first experience with the AoC
* The first approach was based on my colleague's notes on the whiteboard at work; some days later I read the puzzle's description.
* Apart from well known internet sources as [Stack Overflow](https://stackoverflow.com/) and [GeeksForGeeks](https://www.geeksforgeeks.org/) I struggled on my own.
* Coding is in Python as that's the language I'm familiar with: using it since three years now.
* Pretty soon I found a formula that calculates a region's fences price based on amount of garden plots and amount of common sides of adjacent plots.
* The toughest part for me was finding a way to calculate the amount of common sides of adjacent plots.
* Enjoyed solving this puzzle very much.
* I really admire those tackling more than 1 let alone all 25 coding riddles.



In [1]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog
from queue import Queue

### Read in the map with the garden plots
As stated in the puzzle description the map is rectangular and contains >= 2 types of plantes located in garden plots.
* a garden plot contains just 1 type of plant; on the map labeled with a character
* clusters of >= 1 adjacent garden plots with the same type of plant (char) are called regions
* the map may contain > 1 regions of the same type of plant (char)
* a region may enclose a region of another char

The program input of this rectangular map with garden plots is a comma separated CSV file. Example: CSV contents of the puzzle's first garden group and the 2d array after reading in the CSV file:
```
A,A,A,A     [['A' 'A' 'A' 'A']
B,B,C,D      ['B' 'B' 'C' 'D']
B,B,C,C      ['B' 'B' 'C' 'C']
E,E,E,C      ['E' 'E' 'E' 'C']]
```



In [2]:
# read input matrix from csv file
root = tk.Tk()
matrix_file = filedialog.askopenfilename(filetypes=[("CSV matrix files", "*.csv")])
root.destroy()
char_arr = np.loadtxt(matrix_file, delimiter=',', dtype=str)

### Detect regions
The big question to be answered is the total price of fencing all regions on the map. To calculate the perimeter of a region all common sides of adjacent plots are counted. Together with the region's area that count is needed to calculate a region's price.

Detecting regions:
* for a char (type of plant) count the amount of adjacent cells in the 2d matrix; this forms the region
* calculate number of common sides in region

Breadth First Search is used to find all adjacent cells of the given char in the 2d matrix. 

In [3]:
def detect_regions(regions_matrix, char):
    rows, cols = len(regions_matrix), len(regions_matrix[0])
    char_dict_list = []
    visited = [[False] * cols for _ in range(rows)]
    
    directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]
    
    # find all neighbours having element(i, j) == True in the visited matrix (a character of a region)
    def num_neighbour(visited_matrix, i, j):
        count = 0
        if i > 0 and visited_matrix[i - 1][j]: # up
            count += 1
        if j > 0 and visited_matrix[i][j - 1]: # left
            count += 1
        if i < rows - 1 and visited_matrix[i + 1][j]: #down
            count += 1
        if j < cols - 1 and visited_matrix[i][j + 1]: # right
            count += 1
        return count
    
    # breadth first search visits all connected nodes in the matrix to find one or more regions.
    def bfs(queue):
        char_count = adjacent_count = 0 # both needed to later calculate the price of fencing a region
        found_chars= [] # to collect positions of all found characters in a region
        while not queue.empty():
            x = queue.get()
            row, col = map(int, x.split(','))
            # check if row, col without bounds matrix or not already visited or is a different character
            if row < 0 or col < 0 or row >= rows or col >= cols or visited[row][col] or regions_matrix[row][col] != char:
                continue
            
            # char found: set visited True, update char_count and store position
            visited[row][col] = True
            char_count += 1
            found_chars.append((row, col))

            # find adjacent elements for all 4 direction
            for direction in directions:
                new_row, new_col = row + direction[0], col + direction[1]
                queue.put(f'{new_row}, {new_col}')
                
        # finally calculate adjacent_count: sum of common borders of adjacent elements with same char
        total_neighbours = 0
        for pos in found_chars:
            neighbour_count = num_neighbour(visited, pos[0], pos[1])
            total_neighbours += neighbour_count
        adjacent_count = total_neighbours // 2
        char_dict = {'count': char_count, 'adjacent': adjacent_count}
        char_dict_list.append(char_dict)
        
    for i in range(rows):
        for j in range(cols):
            if not visited[i][j] and regions_matrix[i][j] == char:
                queue = Queue()
                queue.put(f'{i}, {j}')
                bfs(queue)

    return char_dict_list

### Price calculation of a region
Price of fencing a region: multiply area by it's perimeter. The perimeter is the fences count that is calculated as follows:
```
count * 4 - adjacent * 2
```
where `count` is the area (number of adjacent cells in a region) and `adjacent` is the total of common sides between adjacent cells.

The function below is for all detected regions of a certain char.

In [4]:
# formula fences_count: count * 4 - adjacent * 2
def price_calc(dict_list):
    price = 0
    for region in dict_list:
        fences_count = region["count"] * 4 - region["adjacent"] * 2
        price += region["count"] * fences_count
    return price

### Main
Print the input, start the calculation and print the results

In [5]:
# MAIN unique chars in char_arr
print(f'Input garden plots map: {os.path.basename(matrix_file)}\n')
total_price = 0
for char in np.unique(char_arr):
    regions_dict_list = detect_regions(char_arr, char)
    print(f'Adjacent {char} regions: {regions_dict_list}')
    region_price = price_calc(regions_dict_list)
    print(f'price region(s) {char}: {region_price}')
    total_price += region_price
print(f'\nTotal fencing price: {total_price}')

Input garden plots map: garden_group_3.csv

Adjacent C regions: [{'count': 14, 'adjacent': 14}, {'count': 1, 'adjacent': 0}]
price region(s) C: 396
Adjacent E regions: [{'count': 13, 'adjacent': 17}]
price region(s) E: 234
Adjacent F regions: [{'count': 10, 'adjacent': 11}]
price region(s) F: 180
Adjacent I regions: [{'count': 4, 'adjacent': 4}, {'count': 14, 'adjacent': 17}]
price region(s) I: 340
Adjacent J regions: [{'count': 11, 'adjacent': 12}]
price region(s) J: 220
Adjacent M regions: [{'count': 5, 'adjacent': 4}]
price region(s) M: 60
Adjacent R regions: [{'count': 12, 'adjacent': 15}]
price region(s) R: 216
Adjacent S regions: [{'count': 3, 'adjacent': 2}]
price region(s) S: 24
Adjacent V regions: [{'count': 13, 'adjacent': 16}]
price region(s) V: 260

Total fencing price: 1930
